### Download organisms/Family data from Norine

In [1]:
##import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

In [2]:
##get norine id
data = pd.read_excel('ids.xlsx', header=0)
data.head()

,CPid,NORINE
0,CP00674,NOR00533
1,CP00675,NOR00656
2,CP00680,NOR00519
3,CP00687,NOR00547
4,CP00689,NOR00530


In [3]:
url = 'https://norine.univ-lille.fr/norine/rest/id/xml/'
for nor in data['NORINE']:
    if not os.path.exists("xmls/"+nor+".xml"):
        #print(nor)
        response = requests.get(url+nor)

        if response.status_code == 200:
            with open("xmls/"+nor+".xml", "wb") as file:
                file.write(response.content)
        else:
            print("请求失败！状态码：%d" % response.status_code)

In [4]:
ORGANISMS = pd.DataFrame(data=None, columns=['NORINE', 'ORGANISMS', 'TAXON ID'])
norids = []
organisms = []
taxids = []

for nor in data['NORINE']:
    peptide_xml = BeautifulSoup(open("xmls/"+nor+".xml"), 'xml')
    organism_list = peptide_xml.find_all('organism')
    
    #organisms = []
    #taxids = []
    if len(organism_list) > 0:
        for child in organism_list:
            organism = np.NAN
            taxid = np.NAN
            if child.find('name') is not None: organism = child.find('name').text
            if child.taxid is not None: taxid = child.taxid.text
            #organisms.append(organism)
            #taxids.append(taxid)
            #norids.append(nor)
            ORGANISMS = pd.concat([ORGANISMS, 
                                  pd.DataFrame([[nor, organism, taxid]], 
                                               columns=['NORINE', 'ORGANISMS', 'TAXON ID'])],
                                  axis=0, ignore_index=True
                                 )
    else:
        ORGANISMS = pd.concat([ORGANISMS, 
                               pd.DataFrame([[nor, np.NAN, np.NAN]], 
                                            columns=['NORINE', 'ORGANISMS', 'TAXON ID'])],
                              axis=0, ignore_index=True
                             )

In [5]:
#ORGANISMS = pd.DataFrame({'NORINE':norids, 'ORGANISMS':organisms, 'TAXON ID':taxids})
ORGANISMS.head()

,NORINE,ORGANISMS,TAXON ID
0,NOR00533,Phakellia sp.,NaN
1,NOR00656,Bacillus cereus,1396
2,NOR00656,Bacillus weihenstephanensis,86662
3,NOR00519,Axinella,12958
4,NOR00547,Hymeniacidon sp.,NaN


In [6]:
ORGANISMS.to_excel('ORGANISMS.xlsx', index=False, header=True)

In [10]:
NORINE_ORGANISMS = pd.merge(data, ORGANISMS, how='left')
NORINE_ORGANISMS.head()

,CPid,NORINE,ORGANISMS,TAXON ID
0,CP00674,NOR00533,Phakellia sp.,NaN
1,CP00675,NOR00656,Bacillus cereus,1396
2,CP00675,NOR00656,Bacillus weihenstephanensis,86662
3,CP00680,NOR00519,Axinella,12958
4,CP00687,NOR00547,Hymeniacidon sp.,NaN


In [11]:
len(NORINE_ORGANISMS)

765

In [12]:
NORINE_ORGANISMS.to_excel('NORINE_ORGANISMS.xlsx', index=False, header=True)

In [14]:
## Family
FAMILY = pd.DataFrame(data=None, columns=['NORINE', 'Family'])

for nor in data['NORINE']:
    peptide_xml = BeautifulSoup(open("xmls/"+nor+".xml"), 'xml')
    family_list = peptide_xml.find_all('family')
 
    if len(family_list) > 0:
        for child in family_list:
            family = child.text
            FAMILY = pd.concat([FAMILY, 
                                  pd.DataFrame([[nor, family]], 
                                               columns=['NORINE', 'Family'])],
                                  axis=0, ignore_index=True
                                 )
    else:
        FAMILY = pd.concat([FAMILY, 
                               pd.DataFrame([[nor, np.NAN]], 
                                            columns=['NORINE', 'Family'])],
                              axis=0, ignore_index=True
                             )
FAMILY.head()

,NORINE,Family
0,NOR00533,phakellistatin
1,NOR00656,cereulide
2,NOR00519,axinastatin
3,NOR00547,hymenistatin
4,NOR00530,phakellistatin


In [15]:
FAMILY.to_excel('FAMILY.xlsx', index=False, header=True)